In [1]:
import httpx
import numpy as np
import orjson
import pandas as pd
from scipy import stats

SEED = 42

BASE = "russia-12-25-baseline-v2"
EXP = "russia-12-25-tariff-change-no-dpx-v4"

URL = "https://drive.google.com/uc?id=1SQsgCXK6CiaMteeRNiWk3sZFKc9FsBg2"

## Load

In [2]:
def load_jsonl(url: str) -> list[dict]:
    rows = []
    with (
        httpx.Client(timeout=30.0, follow_redirects=True) as client,
        client.stream(method="GET", url=url) as response,
    ):
        response.raise_for_status()
        for line in response.iter_lines():
            if not line:
                continue
            row = orjson.loads(line)
            rows.append(row)
    return rows


rows = load_jsonl(url=URL)

len(rows)

28830

## Prepare

In [3]:
orders = pd.json_normalize(rows, sep=".")

orders.shape

(28830, 20)

In [4]:
orders_long = orders.explode("its", ignore_index=True)

its_cols = pd.json_normalize(orders_long["its"]).add_prefix("its.")
items = pd.concat(
    [
        orders_long.drop(columns=["its"]),
        its_cols,
    ],
    axis="columns",
)

items.shape

(29231, 23)

In [5]:
items["_id"] = items["_id.exp"] + items["_id.o"]

items["im.rev.price"] = items["im.rev.price"].div(1_000_000)
items["fm.rev.price"] = items["fm.rev.price"].div(1_000_000)

items["im.t"] = pd.to_datetime(items["im.t"])
items["fm.t"] = pd.to_datetime(items["fm.t"])

In [6]:
items.dtypes

_id.exp                               str
_id.o                                 str
fm.ch                                 str
fm.pid                                str
fm.pw                             float64
fm.rev.price                      float64
fm.st                                 str
fm.t                  datetime64[us, UTC]
fm.w                              float64
im.ch                                 str
im.pid                                str
im.rev.price                      float64
im.st                                 str
im.t                  datetime64[us, UTC]
im.w                              float64
rcv.city                              str
rcv.countryCode                       str
rcv.id                                str
rcv.zipCode                           str
its.dangerousKinds                 object
its.externalId                        str
its.qty                             int64
its.uw                            float64
_id                               

## Explore

In [7]:
# нет пропущенных значений

items.isna().sum(axis="columns").sum()

np.int64(0)

In [8]:
# число уникальных заказов, товаров, пользователей

cols = [
    "_id.o",
    "its.externalId",
    "rcv.id",
]

summary = items[cols].nunique()
summary.name = "nunique"
summary.index.name = "metrics"

summary.reset_index()

,metrics,nunique
0,_id.o,15029
1,its.externalId,15104
2,rcv.id,4253


In [9]:
# число уникальных заказов, товаров, пользователей по группам

cols = [
    "_id.o",
    "its.externalId",
    "rcv.id",
]

items.groupby("_id.exp")[cols].nunique()

,_id.o,its.externalId,rcv.id
_id.exp,,,
russia-12-25-baseline-v2,15029,15104,4253
russia-12-25-tariff-change-no-dpx-v4,13801,13864,3973


In [10]:
# асимметричный diff заказов, товаров, пользователей между группами

cols = [
    "_id.o",
    "its.externalId",
    "rcv.id",
]

rows = []

for col in cols:
    base_mask = items["_id.exp"].eq(BASE)
    exp_mask = items["_id.exp"].eq(EXP)

    base = set(items.loc[base_mask, col])
    exp = set(items.loc[exp_mask, col])

    rows.append(
        {
            "metrics": col,
            "base_minus_exp": len(base - exp),
            "exp_minus_base": len(exp - base),
        }
    )

pd.DataFrame(data=rows).set_index(keys="metrics")

,base_minus_exp,exp_minus_base
metrics,,
_id.o,1228,0
its.externalId,1240,0
rcv.id,280,0


## Фильтрация для идентичности групп

In [11]:
# изначально количество данных в группах разное

items.groupby("_id.exp")["_id"].count().reset_index()

,_id.exp,_id
0,russia-12-25-baseline-v2,15238
1,russia-12-25-tariff-change-no-dpx-v4,13993


In [12]:
# фильтруем лишние данные

base_mask = items["_id.exp"].eq(BASE)
base = set(items.loc[base_mask, "rcv.id"])

exp_mask = items["_id.exp"].eq(EXP)
exp = set(items.loc[exp_mask, "rcv.id"])

extra = base - exp

# переупорядочиваем колонки

ordered_cols = [
    "_id",
    "_id.exp",
    "_id.o",
    "im.pid",
    "fm.pid",
    "im.ch",
    "fm.ch",
    "im.rev.price",
    "fm.rev.price",
    "im.w",
    "fm.w",
    "fm.pw",
    "im.st",
    "fm.st",
    "im.t",
    "fm.t",
    "its.externalId",
    "its.qty",
    "its.uw",
    "its.dangerousKinds",
    "rcv.id",
    "rcv.city",
    "rcv.zipCode",
    "rcv.countryCode",
]

mask = ~items["rcv.id"].isin(extra)
items_filtered = items.loc[mask, ordered_cols].copy()

items_filtered.shape

(27986, 24)

In [13]:
# проверяем после фильтрации
# асимметричный diff заказов, товаров, пользователей между группами

cols = [
    "_id.o",
    "its.externalId",
    "rcv.id",
]

rows = []

for col in cols:
    base_mask = items_filtered["_id.exp"].eq(BASE)
    base = set(items_filtered.loc[base_mask, col])

    exp_mask = items_filtered["_id.exp"].eq(EXP)
    exp = set(items_filtered.loc[exp_mask, col])

    rows.append(
        {
            "metrics": col,
            "base_minus_exp": len(base - exp),
            "exp_minus_base": len(exp - base),
        }
    )

pd.DataFrame(data=rows).set_index(keys="metrics")

,base_minus_exp,exp_minus_base
metrics,,
_id.o,0,0
its.externalId,0,0
rcv.id,0,0


## Примеры

In [14]:
# одна и таже посылка в разных группах

same_parcel_but_diff_groups = [
    "694118cbe263c8b2bb438e94",
    "694161dde6c0cd3557ba3e82",
]

mask = items_filtered["fm.pid"].isin(same_parcel_but_diff_groups)
parcel_example = items_filtered.loc[mask]

In [15]:
# заказы одной и той же посылки в разных группах

cols = items_filtered.columns
parcel_cols = cols[cols.str.contains("_id|im|fm")]

parcel_example[parcel_cols].groupby("_id").agg("first")

# fmt: off
(
    parcel_example[parcel_cols]
    .groupby("_id")
    .agg("first")
    .reset_index()
    .drop("_id", axis="columns")
)
# fmt: on

,_id.exp,_id.o,im.pid,fm.pid,im.ch,fm.ch,im.rev.price,fm.rev.price,im.w,fm.w,fm.pw,im.st,fm.st,im.t,fm.t
0,russia-12-25-baseline-v2,901ZZN5QXL,694118cbe263c8b2bb438e9b,694118cbe263c8b2bb438e94,ChinaPost-CN-RU-RM-EP-ROAD-D,ChinaPost-CN-RU-RM-EP-ROAD-D,69.811529,55.036492,0.4090,0.416,0.831,RM,RM,2025-10-22 07:57:59.938000+00:00,2025-10-25 08:51:44.589000+00:00
1,russia-12-25-baseline-v2,LEG336YXM9,694118cbe263c8b2bb438e9c,694118cbe263c8b2bb438e94,ChinaPost-CN-RU-RM-EP-ROAD-D,ChinaPost-CN-RU-RM-EP-ROAD-D,66.212578,47.940473,0.3340,0.335,0.831,RM,RM,2025-10-22 05:32:32.326000+00:00,2025-10-25 08:51:44.589000+00:00
2,russia-12-25-baseline-v2,LNM6612G59,694118cbe263c8b2bb438e9d,694118cbe263c8b2bb438e94,DPX-CN-RU-RM-PS,ChinaPost-CN-RU-RM-EP-ROAD-D,30.554101,5.531447,0.0285,0.028,0.831,RM,RM,2025-10-22 05:31:09.328000+00:00,2025-10-25 08:51:44.589000+00:00
3,russia-12-25-baseline-v2,VM2NNY6N2V,694118cbe263c8b2bb438e9e,694118cbe263c8b2bb438e94,DPX-CN-RU-RM-PS,ChinaPost-CN-RU-RM-EP-ROAD-D,35.438922,8.305823,0.0490,0.052,0.831,RM,RM,2025-10-22 07:55:58.953000+00:00,2025-10-25 08:51:44.589000+00:00
4,russia-12-25-tariff-change-no-dpx-v4,901ZZN5QXL,694161dde6c0cd3557ba3e89,694161dde6c0cd3557ba3e82,ChinaPost-CN-RU-RM-EP-ROAD-D,ChinaPost-CN-RU-RM-EP-ROAD-D,69.811529,55.036492,0.4090,0.416,0.831,RM,RM,2025-10-22 07:57:59.938000+00:00,2025-10-25 08:51:44.589000+00:00
5,russia-12-25-tariff-change-no-dpx-v4,LEG336YXM9,694161dde6c0cd3557ba3e8a,694161dde6c0cd3557ba3e82,ChinaPost-CN-RU-RM-EP-ROAD-D,ChinaPost-CN-RU-RM-EP-ROAD-D,66.212578,47.940473,0.3340,0.335,0.831,RM,RM,2025-10-22 05:32:32.326000+00:00,2025-10-25 08:51:44.589000+00:00
6,russia-12-25-tariff-change-no-dpx-v4,LNM6612G59,694161dde6c0cd3557ba3e8b,694161dde6c0cd3557ba3e82,ChinaPost-CN-RU-RM-EP-ROAD-D,ChinaPost-CN-RU-RM-EP-ROAD-D,35.416989,5.531447,0.0285,0.028,0.831,RM,RM,2025-10-22 05:31:09.328000+00:00,2025-10-25 08:51:44.589000+00:00
7,russia-12-25-tariff-change-no-dpx-v4,VM2NNY6N2V,694161dde6c0cd3557ba3e8c,694161dde6c0cd3557ba3e82,ChinaPost-CN-RU-RM-EP-ROAD-D,ChinaPost-CN-RU-RM-EP-ROAD-D,37.021976,8.305823,0.0490,0.052,0.831,RM,RM,2025-10-22 07:55:58.953000+00:00,2025-10-25 08:51:44.589000+00:00


In [16]:
# получатель одной и той же посылки в разных группах

cols = items_filtered.columns
receiver_cols = cols[cols.str.contains("_id|rcv")]

# fmt: off
(
    parcel_example[receiver_cols]
    .groupby("_id")
    .agg("first")
    .reset_index()
    .drop("_id", axis="columns")
)
# fmt: on

,_id.exp,_id.o,rcv.id,rcv.city,rcv.zipCode,rcv.countryCode
0,russia-12-25-baseline-v2,901ZZN5QXL,67d6b833b1dced358223ab5c,Бикин,682972,RU
1,russia-12-25-baseline-v2,LEG336YXM9,67d6b833b1dced358223ab5c,Бикин,682972,RU
2,russia-12-25-baseline-v2,LNM6612G59,67d6b833b1dced358223ab5c,Бикин,682972,RU
3,russia-12-25-baseline-v2,VM2NNY6N2V,67d6b833b1dced358223ab5c,Бикин,682972,RU
4,russia-12-25-tariff-change-no-dpx-v4,901ZZN5QXL,67d6b833b1dced358223ab5c,Бикин,682972,RU
5,russia-12-25-tariff-change-no-dpx-v4,LEG336YXM9,67d6b833b1dced358223ab5c,Бикин,682972,RU
6,russia-12-25-tariff-change-no-dpx-v4,LNM6612G59,67d6b833b1dced358223ab5c,Бикин,682972,RU
7,russia-12-25-tariff-change-no-dpx-v4,VM2NNY6N2V,67d6b833b1dced358223ab5c,Бикин,682972,RU


In [17]:
# товары одно и того же заказа в разных группах

cols = items_filtered.columns
item_cols = cols[cols.str.contains("_id.|its")]

order_example = "LEG336YXM9"

mask = items_filtered["_id.o"].eq(order_example)
items_filtered.loc[mask, item_cols]

,_id.exp,_id.o,its.externalId,its.qty,its.uw,its.dangerousKinds
1183,russia-12-25-baseline-v2,LEG336YXM9,7P4EXXN3,1,0.14275,[0]
1184,russia-12-25-baseline-v2,LEG336YXM9,J59GVVY6,1,0.04075,[0]
1185,russia-12-25-baseline-v2,LEG336YXM9,3N9L88QN,1,0.11975,[4]
1186,russia-12-25-baseline-v2,LEG336YXM9,X65VMMG4,1,0.03075,[0]
20687,russia-12-25-tariff-change-no-dpx-v4,LEG336YXM9,7P4EXXN3,1,0.14275,[0]
20688,russia-12-25-tariff-change-no-dpx-v4,LEG336YXM9,J59GVVY6,1,0.04075,[0]
20689,russia-12-25-tariff-change-no-dpx-v4,LEG336YXM9,3N9L88QN,1,0.11975,[4]
20690,russia-12-25-tariff-change-no-dpx-v4,LEG336YXM9,X65VMMG4,1,0.03075,[0]


## Агрегация данных

In [18]:
def is_dangerous(dangerous_kinds: list) -> bool:
    """
    если [0] значит безопасен, иначе нет
    """
    return dangerous_kinds != [0]


agg_mapping = {
    "im.pid": "first",
    "fm.pid": "first",
    "im.ch": "first",
    "fm.ch": "first",
    "im.rev.price": "first",
    "fm.rev.price": "first",
    "im.w": "first",
    "fm.w": "first",
    "fm.pw": "first",
    "im.st": "first",
    "fm.st": "first",
    "im.t": "first",
    "fm.t": "first",
    "its.externalId": lambda order: sorted(order.tolist()),
    "its.qty": "sum",
    "its.uw": "sum",
    "its.dangerousKinds": lambda order: sorted(set(order.explode().tolist())),
    "rcv.id": "first",
    "rcv.city": "first",
    "rcv.zipCode": "first",
    "rcv.countryCode": "first",
}

rename_mapping = {
    "its.externalId": "its.externalId.list",
    "its.qty": "its.qty.sum",
    "its.uw": "its.uw.sum",
    "its.dangerousKinds": "its.dangerousKinds.list",
}

# fmt: off
orders = (
    items_filtered
    .groupby(["_id.exp", "_id.o"])
    .agg(agg_mapping)
    .reset_index()
    .rename(rename_mapping, axis="columns")
)
# fmt: on

orders["its.is_dangerous"] = orders["its.dangerousKinds.list"].apply(is_dangerous)

base_mask = orders["_id.exp"].eq(BASE)
base_orders = orders.loc[base_mask].copy().drop("_id.exp", axis="columns")

exp_mask = orders["_id.exp"].eq(EXP)
exp_orders = orders.loc[exp_mask].copy().drop("_id.exp", axis="columns")


orders.shape, base_orders.shape, exp_orders.shape

((27602, 24), (13801, 23), (13801, 23))

In [19]:
agg_mapping = {
    "_id.o": "count",
    "im.rev.price": "sum",
    "fm.rev.price": "sum",
    "fm.ch": "first",
    "fm.pw": "first",
    "fm.st": "first",
    "its.is_dangerous": "sum",
    "rcv.city": "first",
    "rcv.zipCode": "first",
    "rcv.countryCode": "first",
}

rename_mapping = {
    "_id.o": "order_count",
    "im.rev.price": "im.rev",
    "fm.rev.price": "fm.rev",
}

parcels = (
    orders.groupby(["_id.exp", "fm.pid"])
    .agg(agg_mapping)
    .reset_index()
    .rename(rename_mapping, axis="columns")
)

base_mask = parcels["_id.exp"].eq(BASE)
base_parcels = parcels.loc[base_mask].copy().drop("_id.exp", axis="columns")

exp_mask = parcels["_id.exp"].eq(EXP)
exp_parcels = parcels.loc[exp_mask].copy().drop("_id.exp", axis="columns")


parcels.shape, base_parcels.shape, exp_parcels.shape

((9077, 12), (4543, 11), (4534, 11))

In [20]:
# парные группы сравнений можно выделить только на уровне заказов, т.к. есть _id.o
# группы на уровне посылок непарные, т.к. все посылки стали разными

order_comparison = pd.merge(
    left=base_orders,
    right=exp_orders,
    on="_id.o",
    suffixes=(".base", ".exp"),
    how="inner",
)

order_comparison.shape

(13801, 45)

## Проверка инвариантности признаков между группами

In [21]:
# заказы идентичны

set(base_orders["_id.o"]) == set(exp_orders["_id.o"])

True

In [22]:
# идентичночность метрик "im", "fm", "its", "rcv"

cols = base_orders.columns
keys = ["im", "fm", "its", "rcv"]

metrics = []
for key in keys:
    elems = cols[cols.str.contains(key)].tolist()
    metrics.extend(elems)

# fmt: off
summary = pd.Series(
    data=metrics,
    index=metrics,
    name="is_equal",
)
summary.index.name = "metric"

summary = summary.apply(
    lambda c: (order_comparison[f"{c}.base"] == order_comparison[f"{c}.exp"]).all(),
)
# fmt: on

summary.reset_index()

,metric,is_equal
0,im.pid,False
1,im.ch,False
2,im.rev.price,False
3,im.w,True
4,im.st,True
5,im.t,True
6,fm.pid,False
7,fm.ch,False
8,fm.rev.price,False
9,fm.w,True


**1. Группы между собой**

идентичны по

- _id.o - сделанным заказам

**2. Данные заказов до консолидации (при заказе)**

идентичны между группами

- im.st - тип доставки

- im.t - время создания заказа

- im.w - предсказанный вес

различаются из-за другой модели выбора канала

- im.pid - изначальный айди посылки

- im.ch - изначальный канал доставки

- im.rev.price - изначальная стоимость доставки

**3. Данные заказов после консолидации (на складе)**

идентичны между группа т.к. не зависят от модели выбора канала

- fm.st - тип доставки

- fm.w - фактический вес

различаются из-за другой модели выбора канала

- fm.t - время прибытия на склад

- fm.pw - вес консолидированной посылки

- fm.pid - айди посылки

- fm.ch - финальный канал доставки

- fm.rev.price - финальная стоимость доставки

**4. Данные товаров в заказе**

идентичны между группами

- its.externalId - по SKU

- its.qty - по количеству

- its.uw - по предсказанному весу товара

- its.dangerousKinds - по набору классов опасности

**5. Данные пользователей**

идентичны между группами

- rcv.id - айди пользователя

- rcv.zipCode - зип код пользователя

- rcv.city - город доставки

- rcv.countryCode - страна доставки

## Анализ (квази) эксперимента

### Единица анализа

- **Заказ (_id.o)** — деньги, SLA, распределение каналов
- **Посылка (fm.pid)** — консолидация посылок, вес посылок

### Гипотезы - на что может повлиять отключение канала DPX

#### Экономика

- **H1.** Затраты бизнеса:`fm.rev.price` - по заказам

- **H2.** Цена доставки: `im.rev.price` - по заказам

- **H3.** Прибыль бизнеса: `profit = fm.rev.price − im.rev.price` - по заказам

#### Операционка

- **H4.** Сроки до склада: `fm.t` - mean_delta, p90_delta - по заказам

- **H5.** Консолидация посылок: `n_parcels` на 1k заказов - по посылкам

- **h7.** Вес посылок: `fm.pw` - mean, p90 - по посылкам

#### Структура логистического потока

- **h7.** Изменение долей каналов: топ-5 каналов по доле заказов (`fm.ch`) - по заказам

- **H7.** Миграция заказов между каналами: диаграмма потока baseline → noDPX - по заказам

- **H8.** Концентрация распределения по каналам (`HHI` - индекса Херфиндаля-Хиршмана) - по заказам

### Сегменты:

- По весу заказов `fm.w`
- По количеству товаров в заказах `its.qty.sum`
- По опасности товаров в заказах `dangerous vs non-dangerous`
- По городу пользователя `rcv.city`

можно также
- По весу посылки `fm.pw`
- По количеству заказов в посылке `single-order vs multi-order`
- По логистическому каналу заказа `fm.ch`
- По типу отслеживания заказа `fm.st`
- и другие

### Экономика

In [23]:
def calculate_bootstrap_ci(
    data: pd.Series,
    n_resamples: int = 10000,
    confidence_level: float = 0.95,
) -> tuple[float, float]:
    res = stats.bootstrap(
        (data.values,),
        statistic=np.mean,
        n_resamples=n_resamples,
        confidence_level=confidence_level,
        random_state=SEED,
        method="percentile",
    )
    return res.confidence_interval.low, res.confidence_interval.high

In [24]:
order_comparison["delta_cost"] = (
    order_comparison["fm.rev.price.exp"] - order_comparison["fm.rev.price.base"]
)

order_comparison["delta_price"] = (
    order_comparison["im.rev.price.exp"] - order_comparison["im.rev.price.base"]
)

profit_base = order_comparison["fm.rev.price.base"] - order_comparison["im.rev.price.base"]
profit_exp = order_comparison["fm.rev.price.exp"] - order_comparison["im.rev.price.exp"]
order_comparison["delta_margin"] = profit_exp - profit_base

delta_cost = order_comparison["delta_cost"]
delta_price = order_comparison["delta_price"]
delta_margin = order_comparison["delta_margin"]

ci_lower_h1, ci_upper_h1 = calculate_bootstrap_ci(data=delta_cost)
ci_lower_h2, ci_upper_h2 = calculate_bootstrap_ci(data=delta_price)
ci_lower_h3, ci_upper_h3 = calculate_bootstrap_ci(data=delta_margin)

h1_summary = pd.DataFrame(
    [
        {
            "Hypothesis": "H1: delta cost",
            "Description": "Затраты бизнеса",
            "Mean": f"{delta_cost.mean():.4f}",
            "Median": f"{delta_cost.median():.4f}",
            "Total": f"{delta_cost.sum():.2f}",
            "CI 95% Lower": f"{ci_lower_h1:.4f}",
            "CI 95% Upper": f"{ci_upper_h1:.4f}",
            "Sample Size": f"{delta_cost.size}",
            "Significant": "*" if ci_lower_h1 > 0 or ci_upper_h1 < 0 else "-",
        }
    ]
)

h2_summary = pd.DataFrame(
    [
        {
            "Hypothesis": "H2: delta price",
            "Description": "Цена доставки",
            "Mean": f"{delta_price.mean():.4f}",
            "Median": f"{delta_price.median():.4f}",
            "Total": f"{delta_price.sum():.2f}",
            "CI 95% Lower": f"{ci_lower_h2:.4f}",
            "CI 95% Upper": f"{ci_upper_h2:.4f}",
            "Sample Size": f"{delta_price.size}",
            "Significant": "*" if ci_lower_h2 > 0 or ci_upper_h2 < 0 else "-",
        }
    ]
)

h3_summary = pd.DataFrame(
    [
        {
            "Hypothesis": "H3: delta margin",
            "Description": "Прибыль бизнеса",
            "Mean": f"{delta_margin.mean():.4f}",
            "Median": f"{delta_margin.median():.4f}",
            "Total": f"{delta_margin.sum():.2f}",
            "CI 95% Lower": f"{ci_lower_h3:.4f}",
            "CI 95% Upper": f"{ci_upper_h3:.4f}",
            "Sample Size": f"{delta_margin.size}",
            "Significant": "*" if ci_lower_h3 > 0 or ci_upper_h3 < 0 else "-",
        },
    ]
)

pd.concat(
    [h1_summary, h2_summary, h3_summary],
)

,Hypothesis,Description,Mean,Median,Total,CI 95% Lower,CI 95% Upper,Sample Size,Significant
0,H1: delta cost,Затраты бизнеса,0.9835,0.0000,13572.88,0.9155,1.0523,13801,*
0,H2: delta price,Цена доставки,1.9128,0.0000,26398.49,1.8541,1.9734,13801,*
0,H3: delta margin,Прибыль бизнеса,-0.9293,0.0000,-12825.62,-1.0069,-0.8529,13801,*


### Операционка

In [25]:
order_comparison["delta_time"] = (
    (order_comparison["fm.t.exp"] - order_comparison["fm.t.base"]).dt.total_seconds().div(3600)
)

delta_time = order_comparison["delta_time"]

ci_lower_h4, ci_upper_h4 = calculate_bootstrap_ci(data=delta_time)

h4_summary = pd.DataFrame(
    [
        {
            "Hypothesis": "H4: delta delivery time",
            "Description": "Изменение срока доставки до склада (в часах)",
            "Mean": f"{delta_time.mean():.4f}",
            "Median": f"{delta_time.median():.4f}",
            "Std": f"{delta_time.std():.4f}",
            "P90": f"{delta_time.quantile(0.90):.4f}",
            "P95": f"{delta_time.quantile(0.95)}",
            "CI 95% Lower": f"{ci_lower_h3:.4f}",
            "CI 95% Upper": f"{ci_upper_h3:.4f}",
            "Sample Size": f"{delta_time.size}",
            "Significant": "*" if ci_lower_h4 > 0 or ci_upper_h4 < 0 else "-",
        }
    ]
)

h4_summary

,Hypothesis,Description,Mean,Median,Std,P90,P95,CI 95% Lower,CI 95% Upper,Sample Size,Significant
0,H4: delta delivery time,Изменение срока доставки до склада (в часах),0.2221,0.0000,8.0676,0.0000,0.0,-1.0069,-0.8529,13801,*


In [26]:
n_orders_base = base_orders.size
n_orders_exp = exp_orders.size
n_parcels_base = base_parcels.size
n_parcels_exp = exp_parcels.size

parcels_per_1k_base = n_parcels_base / n_orders_base * 1000
parcels_per_1k_exp = n_parcels_exp / n_orders_exp * 1000

delta_parcels_per_1k = parcels_per_1k_exp - parcels_per_1k_base
change_pct = (delta_parcels_per_1k / parcels_per_1k_base) * 100

h5_summary = pd.DataFrame(
    [
        {
            "Group": "Baseline",
            "Orders": n_orders_base,
            "Parcels": n_parcels_base,
            "Parcels per 1k Orders": f"{parcels_per_1k_base:.2f}",
            "Delta": "-",
            "Delta %": "-",
        },
        {
            "Group": "Experiment (noDPX)",
            "Orders": n_orders_exp,
            "Parcels": n_parcels_exp,
            "Parcels per 1k Orders": f"{parcels_per_1k_exp:.2f}",
            "Delta": f"{delta_parcels_per_1k:+.2f}",
            "Delta %": f"{change_pct:+.2f}%",
        },
    ]
)

h5_summary

,Group,Orders,Parcels,Parcels per 1k Orders,Delta,Delta %
0,Baseline,317423,49973,157.43,-,-
1,Experiment (noDPX),317423,49874,157.12,-0.31,-0.20%


In [27]:
pw_base = base_parcels["fm.pw"]
pw_exp = exp_parcels["fm.pw"]

ci_lower_base, ci_upper_base = calculate_bootstrap_ci(data=pw_base)
ci_lower_exp, ci_upper_exp = calculate_bootstrap_ci(data=pw_exp)

h6_summary = pd.DataFrame(
    [
        {
            "Group": "Baseline",
            "Mean (kg)": f"{pw_base.mean():.4f}",
            "Median (kg)": f"{pw_base.median():.4f}",
            "Std": f"{pw_base.std():.4f}",
            "P90 (kg)": f"{pw_base.quantile(0.90):.4f}",
            "P95 (kg)": f"{pw_base.quantile(0.95):.4f}",
            "CI 95% Lower": f"{ci_lower_base:.4f}",
            "CI 95% Upper": f"{ci_upper_base:.4f}",
            "Sample Size": f"{pw_base.size}",
            "Significant": "None",
        },
        {
            "Group": "Experiment (noDPX)",
            "Mean (kg)": f"{pw_exp.mean():.4f}",
            "Median (kg)": f"{pw_exp.median():.4f}",
            "Std": f"{pw_exp.std():.4f}",
            "P90 (kg)": f"{pw_exp.quantile(0.90):.4f}",
            "P95 (kg)": f"{pw_exp.quantile(0.95):.4f}",
            "CI 95% Lower": f"{ci_lower_exp:.4f}",
            "CI 95% Upper": f"{ci_upper_exp:.4f}",
            "Sample Size": f"{pw_exp.size}",
            "Significant": "None",
        },
    ]
)

h6_summary

,Group,Mean (kg),Median (kg),Std,P90 (kg),P95 (kg),CI 95% Lower,CI 95% Upper,Sample Size,Significant
0,Baseline,0.7672,0.5490,0.7079,1.6900,2.1419,0.7471,0.7884,4543,None
1,Experiment (noDPX),0.7687,0.5555,0.7002,1.7000,2.1553,0.7481,0.7891,4534,None


### Структура логистического потока

In [28]:
shares_base = order_comparison["fm.ch.base"].value_counts(normalize=True)
shares_exp = order_comparison["fm.ch.exp"].value_counts(normalize=True)

TOP_N = 5

h7_summary = (
    pd.DataFrame(
        {
            "Channel": shares_base.head(TOP_N).index,
            "Baseline Share": shares_base.head(TOP_N).values,
        }
    )
    .merge(
        pd.DataFrame(
            {
                "Channel": shares_exp.head(TOP_N).index,
                "Experiment Share": shares_exp.head(TOP_N).values,
            }
        ),
        on="Channel",
        how="outer",
    )
    .fillna(0)
)

h7_summary["Delta Share"] = h7_summary["Experiment Share"] - h7_summary["Baseline Share"]

h7_summary["Baseline Share"] = h7_summary["Baseline Share"].apply(lambda x: f"{x:.4f}")
h7_summary["Experiment Share"] = h7_summary["Experiment Share"].apply(lambda x: f"{x:.4f}")
h7_summary["Delta Share"] = h7_summary["Delta Share"].apply(lambda x: f"{x:+.4f}")

h7_summary = h7_summary.sort_values("Baseline Share", ascending=False)

h7_summary

,Channel,Baseline Share,Experiment Share,Delta Share
2,ChinaPost-CN-RU-RM-EP-ROAD-D,0.6416,0.8547,+0.2131
4,DPX-CN-RU-RM-epacket-PS,0.1817,0.0001,-0.1816
1,Cainiao-CN-RU-RM-PUDO-DG,0.1333,0.1333,+0.0000
3,DPX-CN-RU-RM-PS,0.0283,0.0000,-0.0283
0,Cainiao-CN-RU-RM-PUDO,0.0119,0.0119,+0.0000


In [29]:
ch_flow = (
    order_comparison.groupby(["fm.ch.base", "fm.ch.exp"], observed=True)
    .size()
    .reset_index(name="count")
)

total_orders = order_comparison.size
ch_flow["share"] = ch_flow["count"] / total_orders
ch_flow["share_pct"] = ch_flow["share"].mul(100).round(2)

TOP_K = 10

ch_flow_top_k = ch_flow.sort_values("count", ascending=False).head(TOP_K).copy()

ch_flow_top_k["count_formatted"] = ch_flow_top_k["count"].apply(lambda x: f"{x:,}")
ch_flow_top_k["share_formatted"] = ch_flow_top_k["share"].apply(lambda x: f"{x:.4f}")

h8_summary = ch_flow_top_k[["fm.ch.base", "fm.ch.exp", "count_formatted", "share_pct"]].rename(
    columns={
        "fm.ch.base": "Channel (Baseline)",
        "fm.ch.exp": "Channel (Experiment)",
        "count_formatted": "Orders",
        "share_pct": "Share %",
    }
)

h8_summary

,Channel (Baseline),Channel (Experiment),Orders,Share %
2,ChinaPost-CN-RU-RM-EP-ROAD-D,ChinaPost-CN-RU-RM-EP-ROAD-D,"8,855",1.31
4,DPX-CN-RU-RM-epacket-PS,ChinaPost-CN-RU-RM-EP-ROAD-D,"2,506",0.37
1,Cainiao-CN-RU-RM-PUDO-DG,Cainiao-CN-RU-RM-PUDO-DG,"1,840",0.27
3,DPX-CN-RU-RM-PS,ChinaPost-CN-RU-RM-EP-ROAD-D,390,0.06
0,Cainiao-CN-RU-RM-PUDO,Cainiao-CN-RU-RM-PUDO,164,0.02
6,DPX-CN-RU-RM-eparcel-PS,ChinaPost-CN-RU-RM-EP-ROAD-D,44,0.01
5,DPX-CN-RU-RM-epacket-PS,DPX-CN-RU-RM-epacket-PS,1,0.00
7,DPX-CN-RU-RMH-PS,ChinaPost-CN-RU-RM-EP-ROAD-D,1,0.00


In [30]:
def calculate_hhi(series: pd.Series) -> float:
    """
    Расчет индекса Херфиндаля-Хиршмана HHI = Σ(share_i)^2
    Значения: 0 (равномерное распределение) до 1 (монополия)
    """
    shares = series.value_counts(normalize=True)
    return (shares**2).sum()


hhi_base = calculate_hhi(series=order_comparison["fm.ch.base"])
hhi_exp = calculate_hhi(series=order_comparison["fm.ch.exp"])
delta_hhi = hhi_exp - hhi_base
change_hhi_pct = (delta_hhi / hhi_base) * 100

h8_summary = pd.DataFrame(
    [
        {
            "Group": "Baseline",
            "HHI": f"{hhi_base:.4f}",
            "Delta": "-",
            "Delta %": "-",
            "Interpretation": "Lower = more distributed",
        },
        {
            "Group": "Experiment (noDPX)",
            "HHI": f"{hhi_exp:.4f}",
            "Delta": f"{delta_hhi:+.4f}",
            "Delta %": f"{change_hhi_pct:+.2f}%",
            "Interpretation": "Higher = more concentrated"
            if delta_hhi > 0
            else "Lower = more distributed",
        },
    ]
)

h8_summary

,Group,HHI,Delta,Delta %,Interpretation
0,Baseline,0.4634,-,-,Lower = more distributed
1,Experiment (noDPX),0.7485,+0.2851,+61.52%,Higher = more concentrated


### Сегменты

In [31]:
# по весу заказов

bins_w = [0, 0.5, 1.0, float("inf")]
labels_w = ["0–0.5 kg", "0.5–1 kg", "1+ kg"]
order_comparison["fm.w.bin"] = pd.cut(order_comparison["fm.w.base"], bins=bins_w, labels=labels_w)

seg_w = (
    order_comparison.groupby("fm.w.bin", observed=True)
    .agg(
        n=("_id.o", "count"),
        mean_delta_cost=("delta_cost", "mean"),
        mean_delta_margin=("delta_margin", "mean"),
        mean_delta_time=("delta_time", "mean"),
    )
    .reset_index()
)

seg_w["share_pct"] = (seg_w["n"] / seg_w["n"].sum() * 100).round(2)

seg_w = seg_w.copy()
seg_w["Orders"] = seg_w["n"].apply(lambda x: f"{x:,}")
seg_w["Share %"] = seg_w["share_pct"]
seg_w["Delta Cost"] = seg_w["mean_delta_cost"].apply(lambda x: f"{x:.4f}")
seg_w["Delta Margin"] = seg_w["mean_delta_margin"].apply(lambda x: f"{x:.4f}")
seg_w["Delta Time (h)"] = seg_w["mean_delta_time"].apply(lambda x: f"{x:.4f}")

seg_w[["fm.w.bin", "Orders", "Share %", "Delta Cost", "Delta Margin", "Delta Time (h)"]].rename(
    columns={"fm.w.bin": "Weight (order)"}
)

,Weight (order),Orders,Share %,Delta Cost,Delta Margin,Delta Time (h)
0,0–0.5 kg,"11,736",85.04,0.2583,-1.6441,0.3054
1,0.5–1 kg,"1,336",9.68,3.2603,3.1603,-0.4691
2,1+ kg,729,5.28,8.4850,3.0828,0.1486


In [32]:
# по количеству товаров в заказах

order_comparison["qty.bin"] = pd.cut(
    order_comparison["its.qty.sum.base"], bins=[0, 1, 2, 10], labels=["1", "2", "3+"]
)

seg_qty = (
    order_comparison.groupby("qty.bin", observed=True)
    .agg(
        n=("_id.o", "count"),
        mean_delta_cost=("delta_cost", "mean"),
        mean_delta_margin=("delta_margin", "mean"),
    )
    .reset_index()
)

seg_qty["share_pct"] = (seg_qty["n"] / seg_qty["n"].sum() * 100).round(2)

seg_qty = seg_qty.copy()
seg_qty["Orders"] = seg_qty["n"].apply(lambda x: f"{x:,}")
seg_qty["Share %"] = seg_qty["share_pct"]
seg_qty["Delta Cost"] = seg_qty["mean_delta_cost"].apply(lambda x: f"{x:.4f}")
seg_qty["Delta Margin"] = seg_qty["mean_delta_margin"].apply(lambda x: f"{x:.4f}")

seg_qty[["qty.bin", "Orders", "Share %", "Delta Cost", "Delta Margin"]].rename(
    columns={"qty.bin": "Items qty"}
)

,Items qty,Orders,Share %,Delta Cost,Delta Margin
0,1,"12,685",91.94,1.0044,-0.9702
1,2,628,4.55,1.3197,-0.0482
2,3+,484,3.51,0.0058,-1.0100


In [33]:
# по опасности товаров в заказах

order_comparison["is_dangerous"] = order_comparison["its.is_dangerous.base"].astype(bool)

seg_dangerous = (
    order_comparison.groupby("is_dangerous")
    .agg(
        n=("_id.o", "count"),
        mean_delta_cost=("delta_cost", "mean"),
        mean_delta_margin=("delta_margin", "mean"),
    )
    .reset_index()
)

seg_dangerous["share_pct"] = (seg_dangerous["n"] / seg_dangerous["n"].sum() * 100).round(2)

# Форматирование
seg_dangerous = seg_dangerous.copy()
seg_dangerous["Category"] = seg_dangerous["is_dangerous"].map(
    {True: "Dangerous", False: "Non-Dangerous"}
)
seg_dangerous["Orders"] = seg_dangerous["n"].apply(lambda x: f"{x:,}")
seg_dangerous["Share %"] = seg_dangerous["share_pct"]
seg_dangerous["Delta Cost"] = seg_dangerous["mean_delta_cost"].apply(lambda x: f"{x:.4f}")
seg_dangerous["Delta Margin"] = seg_dangerous["mean_delta_margin"].apply(lambda x: f"{x:.4f}")

seg_dangerous[["Category", "Orders", "Share %", "Delta Cost", "Delta Margin"]]

,Category,Orders,Share %,Delta Cost,Delta Margin
0,Non-Dangerous,"12,449",90.2,1.0455,-0.8436
1,Dangerous,"1,352",9.8,0.4124,-1.7186


In [34]:
# по городам получателя

TOP_N = 5

top_cities = order_comparison["rcv.city.base"].value_counts().head(TOP_N).index.tolist()
order_comparison["city_segment"] = order_comparison["rcv.city.base"].where(
    order_comparison["rcv.city.base"].isin(top_cities), "Other Cities"
)

seg_city = (
    order_comparison.groupby("city_segment", observed=True)
    .agg(
        n=("_id.o", "count"),
        mean_delta_cost=("delta_cost", "mean"),
        mean_delta_margin=("delta_margin", "mean"),
    )
    .reset_index()
)

seg_city["share_pct"] = (seg_city["n"] / seg_city["n"].sum() * 100).round(2)
seg_city = seg_city.sort_values("n", ascending=False)

# Форматирование
seg_city = seg_city.copy()
seg_city["Orders"] = seg_city["n"].apply(lambda x: f"{x:,}")
seg_city["Share %"] = seg_city["share_pct"]
seg_city["Delta Cost"] = seg_city["mean_delta_cost"].apply(lambda x: f"{x:.4f}")
seg_city["Delta Margin"] = seg_city["mean_delta_margin"].apply(lambda x: f"{x:.4f}")

seg_city[["city_segment", "Orders", "Share %", "Delta Cost", "Delta Margin"]].rename(
    columns={"city_segment": "City"}
)

,City,Orders,Share %,Delta Cost,Delta Margin
0,Other Cities,"11,350",82.24,0.9342,-0.9675
2,Москва,"1,242",9.00,1.2865,-0.5804
4,Санкт-Петербург,727,5.27,1.1917,-0.7988
5,Челябинск,173,1.25,0.8198,-0.8972
1,Екатеринбург,157,1.14,0.5505,-1.9359
3,Пермь,152,1.10,1.8240,-0.5490
